In [14]:
import math

def solve_relationship(relation, values):
    """Solve various relationship types given some values"""
    keys, rel_type = relation
    
    # Get current values for this relationship
    vals = [values.get(key) for key in keys]
    provided = sum(x is not None for x in vals)
    
    # Need at least 2 values for most relationships, 4 for 5-parameter relationships
    min_needed = len(keys) - 1 if len(keys) <= 4 else len(keys) - 1
    if provided < min_needed:
        return {}
    
    result = {}
    
    if rel_type == "reciprocal_sum":  # 1/a = 1/b + 1/c → thin lens equation
        a_key, b_key, c_key = keys
        a, b, c = vals
        
        if a is None: 
            a = 1 / (1/b + 1/c)
        elif b is None: 
            b = 1 / (1/a - 1/c)
        elif c is None: 
            c = 1 / (1/a - 1/b)
            
        result = {a_key: a, b_key: b, c_key: c}
    
    elif rel_type == "reciprocal":  # a = 1/b
        a_key, b_key = keys
        a, b = vals
        
        if a is None: 
            a = 1/b
        elif b is None: 
            b = 1/a
            
        result = {a_key: a, b_key: b}
    
    elif rel_type == "ratio":  # a = b/c
        a_key, b_key, c_key = keys
        a, b, c = vals
        
        if a is None: 
            a = b/c
        elif b is None: 
            b = a * c
        elif c is None: 
            c = b/a
            
        result = {a_key: a, b_key: b, c_key: c}
    
    elif rel_type == "negative_ratio":  # a = -b/c
        a_key, b_key, c_key = keys
        a, b, c = vals
        
        if a is None: 
            a = -b/c
        elif b is None: 
            b = -a * c
        elif c is None: 
            c = -b/a
            
        result = {a_key: a, b_key: b, c_key: c}
    
    elif rel_type == "lens_maker_thin":  # 1/f = (n-1)[1/R1 - 1/R2]
        f_key, n_key, R1_key, R2_key = keys
        f, n, R1, R2 = vals
        
        # Need at least 3 of 4 values
        if provided < 3:
            return {}
        
        if f is None and all(x is not None for x in [n, R1, R2]):
            f = 1 / ((n-1) * (1/R1 - 1/R2))
        elif n is None and all(x is not None for x in [f, R1, R2]):
            n = 1 + 1/(f * (1/R1 - 1/R2))
        elif R1 is None and all(x is not None for x in [f, n, R2]):
            # 1/R1 = 1/[f(n-1)] + 1/R2
            R1 = 1 / (1/(f*(n-1)) + 1/R2)
        elif R2 is None and all(x is not None for x in [f, n, R1]):
            # 1/R2 = 1/R1 - 1/[f(n-1)]
            R2 = 1 / (1/R1 - 1/(f*(n-1)))
        else:
            return {}
            
        result = {f_key: f, n_key: n, R1_key: R1, R2_key: R2}
    
    elif rel_type == "lens_maker_thick":  # 1/f = (n-1)[1/R1 - 1/R2 + (n-1)d/(nR1R2)]
        f_key, n_key, R1_key, R2_key, d_key = keys
        f, n, R1, R2, d = vals
        
        if f is None:
            term1 = 1/R1 - 1/R2
            term2 = (n-1) * d / (n * R1 * R2)
            f = 1 / ((n-1) * (term1 + term2))
        elif n is None:
            # This gets complex for thick lenses, skip for now
            pass
        elif d is None:
            # d = n*R1*R2/((n-1)) * [1/(f(n-1)) - (1/R1 - 1/R2)]
            basic_term = 1/R1 - 1/R2
            thick_correction = 1/(f*(n-1)) - basic_term
            d = (n * R1 * R2 * thick_correction) / (n-1)
        # R1, R2 solving for thick lenses is quite complex, skip for now
            
        result = {f_key: f, n_key: n, R1_key: R1, R2_key: R2, d_key: d}
    
    elif rel_type == "thick_lens_correction":  # Relationship between effective focal length and back focal length
        # f_eff = f_back + d/n (simplified approximation)
        f_eff_key, f_back_key, d_key, n_key = keys
        f_eff, f_back, d, n = vals
        
        if f_eff is None:
            f_eff = f_back + d/n
        elif f_back is None:
            f_back = f_eff - d/n
        elif d is None:
            d = (f_eff - f_back) * n
        elif n is None:
            n = d / (f_eff - f_back)
            
        result = {f_eff_key: f_eff, f_back_key: f_back, d_key: d, n_key: n}
    
    return result

def lens_calculator(f=None, do=None, di=None, m=None, hi=None, ho=None, P=None,
                   n=None, R1=None, R2=None, d=None, f_eff=None, f_back=None):
    """
    Complete lens calculator using structural relationships
    
    Relationships:
    - Thin lens: 1/f = 1/do + 1/di
    - Magnification: m = -di/do = hi/ho  
    - Power: P = 1/f
    - Lens maker (thin): 1/f = (n-1)[1/R1 - 1/R2]
    - Lens maker (thick): 1/f = (n-1)[1/R1 - 1/R2 + (n-1)d/(nR1R2)]
    - Thick lens: f_eff ≈ f_back + d/n
    """
    
    # Define all optical relationships structurally
    relationships = [
        # Basic lens relationships
        (('f', 'do', 'di'), 'reciprocal_sum'),           # 1/f = 1/do + 1/di
        (('P', 'f'), 'reciprocal'),                      # P = 1/f
        (('m', 'di', 'do'), 'negative_ratio'),           # m = -di/do
        (('m', 'hi', 'ho'), 'ratio'),                    # m = hi/ho
        
        # Lens maker's equations
        (('f', 'n', 'R1', 'R2'), 'lens_maker_thin'),     # 1/f = (n-1)[1/R1 - 1/R2]
        (('f', 'n', 'R1', 'R2', 'd'), 'lens_maker_thick'), # Full thick lens maker
        
        # Thick lens corrections
        (('f_eff', 'f_back', 'd', 'n'), 'thick_lens_correction'), # f_eff = f_back + d/n
    ]
    
    values = {
        'f': f, 'do': do, 'di': di, 'm': m, 'hi': hi, 'ho': ho, 'P': P,
        'n': n, 'R1': R1, 'R2': R2, 'd': d, 'f_eff': f_eff, 'f_back': f_back
    }
    provided_count = sum(v is not None for v in values.values())
    
    if provided_count < 2:
        return {**values, 'valid': True, 'sufficient': False}
    
    # Iteratively apply relationships until no more progress
    try:
        max_iterations = 15  # More iterations for complex relationships
        for iteration in range(max_iterations):
            old_values = values.copy()
            
            for relation in relationships:
                result = solve_relationship(relation, values)
                if result:
                    values.update(result)
            
            # Stop if no new values were calculated
            if values == old_values:
                break
        
        # Enhanced validation for all relationships
        tolerance = 1e-9
        valid = True
        
        # Check thin lens equation: 1/f = 1/do + 1/di
        if all(values[k] is not None for k in ['f', 'do', 'di']):
            expected = 1/values['do'] + 1/values['di']
            actual = 1/values['f']
            if abs(expected - actual) > tolerance:
                valid = False
        
        # Check lens maker thin: 1/f = (n-1)[1/R1 - 1/R2]
        if all(values[k] is not None for k in ['f', 'n', 'R1', 'R2']):
            expected = (values['n']-1) * (1/values['R1'] - 1/values['R2'])
            actual = 1/values['f']
            if abs(expected - actual) > tolerance:
                valid = False
        
        # Check lens maker thick: 1/f = (n-1)[1/R1 - 1/R2 + (n-1)d/(nR1R2)]
        if all(values[k] is not None for k in ['f', 'n', 'R1', 'R2', 'd']):
            term1 = 1/values['R1'] - 1/values['R2']
            term2 = (values['n']-1) * values['d'] / (values['n'] * values['R1'] * values['R2'])
            expected = (values['n']-1) * (term1 + term2)
            actual = 1/values['f']
            if abs(expected - actual) > tolerance:
                valid = False
        
        # Check other relationships...
        if all(values[k] is not None for k in ['P', 'f']):
            if abs(values['P'] - 1/values['f']) > tolerance:
                valid = False
        
        if all(values[k] is not None for k in ['m', 'di', 'do']):
            if abs(values['m'] - (-values['di']/values['do'])) > tolerance:
                valid = False
        
        if all(values[k] is not None for k in ['m', 'hi', 'ho']):
            if abs(values['m'] - (values['hi']/values['ho'])) > tolerance:
                valid = False
            
        return {**values, 'valid': valid}
        
    except (ZeroDivisionError, ValueError):
        return {**values, 'valid': False}


# Test cases for advanced lens calculations
def run_tests():
    print("ADVANCED STRUCTURAL LENS CALCULATOR")
    print("=" * 60)
    
    test_cases = [
        # Basic thin lens
        ("Basic thin lens", {'f': 10, 'do': 15}),
        
        # Lens maker's equation (thin lens)
        ("Glass lens design", {'n': 1.5, 'R1': 20, 'R2': -30}),
        
        # Combining lens maker with imaging
        ("Design + imaging", {'n': 1.5, 'R1': 10, 'R2': -10, 'do': 25}),
        
        # Thick lens example
        ("Thick lens", {'n': 1.5, 'R1': 20, 'R2': -20, 'd': 5, 'do': 50}),
        
        # Complex validation
        ("Full validation", {
            'n': 1.6, 'R1': 15, 'R2': -25, 'd': 3, 
            'do': 30, 'hi': 2, 'ho': 1
        }),
        
        # Power and lens maker
        ("Power + design", {'P': 0.1, 'n': 1.5, 'R1': 20}),
        
        # Thick lens corrections
        ("Thick corrections", {'f_back': 18, 'd': 4, 'n': 1.5}),
    ]
    
    for name, params in test_cases:
        print(f"\n{name}:")
        print(f"Input:  {params}")
        result = lens_calculator(**params)
        
        # Clean output - only show calculated values
        output = {k: v for k, v in result.items() if v is not None}
        print(f"Output: {output}")
        
        # Show key results clearly
        if result.get('f') is not None:
            print(f"  → Focal length: {result['f']:.2f}")
        if result.get('di') is not None:
            print(f"  → Image distance: {result['di']:.2f}")
        if result.get('m') is not None:
            print(f"  → Magnification: {result['m']:.2f}×")
        if result.get('valid') is not None:
            print(f"  → Valid: {result['valid']}")

run_tests()

ADVANCED STRUCTURAL LENS CALCULATOR

Basic thin lens:
Input:  {'f': 10, 'do': 15}
Output: {'f': 10, 'do': 15, 'di': 29.999999999999993, 'm': -1.9999999999999996, 'P': 0.1, 'valid': True}
  → Focal length: 10.00
  → Image distance: 30.00
  → Magnification: -2.00×
  → Valid: True

Glass lens design:
Input:  {'n': 1.5, 'R1': 20, 'R2': -30}
Output: {'f': 23.999999999999996, 'P': 0.04166666666666667, 'n': 1.5, 'R1': 20, 'R2': -30, 'd': -0.0, 'valid': True}
  → Focal length: 24.00
  → Valid: True

Design + imaging:
Input:  {'n': 1.5, 'R1': 10, 'R2': -10, 'do': 25}
Output: {'f': 10.0, 'do': 25, 'di': 16.666666666666664, 'm': -0.6666666666666665, 'P': 0.1, 'n': 1.5, 'R1': 10, 'R2': -10, 'd': -0.0, 'valid': True}
  → Focal length: 10.00
  → Image distance: 16.67
  → Magnification: -0.67×
  → Valid: True

Thick lens:
Input:  {'n': 1.5, 'R1': 20, 'R2': -20, 'd': 5, 'do': 50}
Output: {'f': 20.0, 'do': 50, 'di': 33.33333333333333, 'm': -0.6666666666666665, 'P': 0.05, 'n': 1.5, 'R1': 20, 'R2': -20, 